In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import r2_score

In [2]:
# Build a example dataset
data = pd.DataFrame({'a':range(1000), 'b':range(1000,2000), 'y': [i/1000 for i in range(1000)]}).sample(frac=1, random_state=17).reset_index(drop=True)
data

,a,b,y
0,786,1786,0.786
1,499,1499,0.499
2,930,1930,0.930
3,217,1217,0.217
4,803,1803,0.803
...,...,...,...
995,406,1406,0.406
996,390,1390,0.390
997,143,1143,0.143
998,241,1241,0.241


In [3]:
# Split the data into train and test
X = data.drop('y', axis=1)
y = data['y']

In [4]:
X_train, y_train = X[:600], y[:600]
X_val, y_val = X[600:800], y[600:800]
X_test, y_test = X[800:], y[800:]
print(X_train.shape, X_val.shape, X_test.shape)

(600, 2) (200, 2) (200, 2)


In [5]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Keras Model

In [6]:
def build_model(params: dict)-> tf.keras.models.Sequential:
    """
    Build a simple feedforward neural network.
    
    Args:
        params: dict, containing the following keys:
            - input_shape: int, number of features in the input data
            - layers: list of int, number of neurons in each layer
            - activation: list of str, activation function for each layer
    
    Returns:
        model: keras model, the compiled neural network
    """
    
    # Initialize the model
    model = Sequential()
    
    # Add the input layer
    model.add(Dense(params['layers'][0], input_dim=params['input_shape'], activation=params['activation'][0]))
    
    # Add the remaining hidden layers
    if len(params['layers']) > 1:
        for layer, activation in zip(params['layers'][1:], params['activation'][1:]):
            model.add(Dense(layer, activation=activation))
    
    return model

In [7]:
# Build the model
params={
    'input_shape': 2,
    'layers': [64, 32, 16, 1],
    'activation': ['relu', 'relu', 'tanh', 'sigmoid'],
}

In [8]:
# Set random seed
tf.random.set_seed(17)

In [9]:
# Initialize the model
model = build_model(params)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                192       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2817 (11.00 KB)
Trainable params: 2817 (11.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
optimizer = Adam(learning_rate=0.01)
scheduler = LearningRateScheduler(lambda _, lr: lr * 0.9)

In [12]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[r2_score], run_eagerly=True)

In [13]:
# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, start_from_epoch=20)

In [14]:
# Train the model
model.fit(X_train, y_train, validation_data=[X_val, y_val], epochs=100, batch_size=64, callbacks=[early_stopping, scheduler], verbose=1)

Epoch 1/100

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
10/10 [==============================] - 3s 98ms/step - loss: 0.0140 - r2_score: 0.8341 - val_loss: 0.0032 - val_r2_score: 0.9629 - lr: 0.0090
Epoch 2/100
10/10 [==============================] - 1s 94ms/step - loss: 0.0022 - r2_score: 0.9730 - val_loss: 0.0017 - val_r2_score: 0.9813 - lr: 0.0081
Epoch 3/100
10/10 [==============================] - 1s 95ms/step - loss: 0.0011 - r2_score: 0.9865 - val_loss: 9.1584e-04 - val_r2_score: 0.9892 - lr: 0.0073
Epoch 4/100
10/10 [==============================] - 1s 94ms/step - loss: 7.0551e-04 - r2_score: 0.9912 - val_loss: 6.6766e-04 - val_r2_score: 0.9931 - lr: 0.0066
Epoch 5/100
10/10 [==============================] - 1s 94ms/step - loss: 5.2953e-04 - r2_s

In [15]:
# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

7/7 [==============================] - 0s 54ms/step - loss: 1.3642e-04 - r2_score: 0.9986
Accuracy: 99.86
